In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config(path = "config.json")
print(ws)

In [ ]:
from azureml.core.model import Model
#import logging
#logging.basicConfig(level=logging.DEBUG)

# Register model
model0 = Model.register(
    ws, 
    model_name = "readability-model-p0", 
    model_path = "model_0.joblib",
    model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
    model_framework_version="0.23.2",
    description='Predict readability of passage. First part of model.',
    tags={'type': 'regression'}
)
model1 = Model.register(
    ws, 
    model_name = "readability-model-p1", 
    model_path = "model_1.joblib",
    model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
    model_framework_version="0.23.2",
    description='Predict readability of passage. Second part of model.',
    tags={'type': 'regression'}
)

#print(Model.get_model_path(model_name='model.pkl'))

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment(name = "readability-model")
conda_dep = CondaDependencies()
conda_dep.set_python_version("3.7.10")
conda_dep.add_pip_package("https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.3.1/en_core_web_lg-2.3.1.tar.gz")
conda_dep.add_pip_package("azureml-model-management-sdk")
conda_dep.add_pip_package("numpy==1.19.5")
conda_dep.add_pip_package("scikit-learn==0.23.2")
conda_dep.add_pip_package("nltk==3.2.4")
conda_dep.add_pip_package("spacy==2.3.7")
conda_dep.add_pip_package("pandas==1.2.4")
conda_dep.add_pip_package("readability==0.3.1")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "score.py",
)


In [ ]:
# Create a cloud deployment.
# from azureml.core.webservice import AciWebservice
# deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4)

# Create a local deployment, using port 8890 for the web service endpoint
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=8890)

service = Model.deploy(
    ws,
    "readability-model",
    [model1, model0],
    inference_config=dummy_inference_config,
    deployment_config=deployment_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get(uri)
headers = {"Content-Type": "application/json"}
data = {
    "id": "965e592c0",
    "url_legal": "https://www.africanstorybook.org/#",
    "license": "CC BY 4.0",
    "excerpt": "Milka and John are playing in the garden. Her little sister is playing too. Milka is ready to start classes next week and it will be her first term in school. In the morning, Milka gets up early to take a bath. She puts on her school uniform and carries her school bag. Her Mother gives her two thousand shillings for school fees and five hundred shillings for transport. Then, she quickly goes to school. Meanwhile, her big brother stays at home. He is still in his bed and sleeps. Once she grows up and graduates school, Milka dreams to build a beautiful house for her and her family. While she is at school, she is very active and participates in all the activities. The teachers love her attitude. Milka listens carefully to her teacher. Her classmates admire her too, because she is a kind girl. At break time she tries to help other classmates with their practical exercies and homeworks."
}
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.text)

In [ ]:
print(service.get_logs())

In [ ]:
# Remove the models
model0.delete()
model1.delete()

In [ ]:
# Delete the service
service.delete()